In [17]:
# print_function for compatibility with Python 3
from __future__ import print_function
print('print function is ready to serve')

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline

# NumPy for numerical computing
import numpy as np
np.random.seed(123)
import random
random.seed(123)

# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_column', 100)

# sklearn for model training
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import os
import gc
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

print function is ready to serve


In [2]:
import sys
sys.path.append('./utils')

from data import Data
from models import Models
from tags import Tags
tags = Tags()

Using TensorFlow backend.


In [3]:
PLANET_KAGGLE_ROOT = '/data/planet-data/'
if not os.path.exists(PLANET_KAGGLE_ROOT):
    PLANET_KAGGLE_ROOT = '/Users/jiayou/Documents/Kaggle Data/Amazon'

N_TAGS = 17
N_TRAIN = 40479
# N_TRAIN = 10
N_TEST_T = 40669
N_TEST_F = 20522
N_TEST = N_TEST_T + N_TEST_F

In [6]:
val = 4
train_pred = np.load(os.path.join(PLANET_KAGGLE_ROOT, 'ensemble', 'raw_train_pred.v9.f{}.tta8.npy'.format(val)))
y_true = tags.y_train(range(N_TRAIN))

In [7]:
y_true.shape

(40479, 17)

In [15]:
tag_id = 1
feature = train_pred[:,tag_id]
feature = feature.reshape((N_TRAIN, 8))
y = y_true[:,tag_id]
feature[:20]

array([[ 0.99988508,  0.99992394,  0.99992931,  0.9999404 ,  0.99994886,
         0.9999249 ,  0.99991739,  0.99986732],
       [ 0.99981648,  0.99988842,  0.99989939,  0.99991977,  0.99992716,
         0.99997962,  0.99997914,  0.99979585],
       [ 0.99998665,  0.99999285,  0.99999213,  0.99997306,  0.99997568,
         0.99998569,  0.99998927,  0.99998856],
       [ 0.99998236,  0.99998641,  0.99998534,  0.99998748,  0.99998844,
         0.99998963,  0.99998748,  0.9999851 ],
       [ 0.98875463,  0.98906404,  0.99273419,  0.99573368,  0.99392128,
         0.99558973,  0.99535817,  0.99027658],
       [ 0.98680645,  0.97594714,  0.98422468,  0.98863518,  0.98770344,
         0.98292094,  0.97656107,  0.98564434],
       [ 0.99936754,  0.99843389,  0.99904042,  0.99957711,  0.99939501,
         0.99871278,  0.99884206,  0.99948221],
       [ 0.86359537,  0.9050163 ,  0.91166967,  0.8922053 ,  0.8615253 ,
         0.88500553,  0.9287594 ,  0.88375068],
       [ 0.99997354,  0.99997115